This notebook examines the effect of varying the size of the calibration dataset on the mean task loss and financial CVaR tail risk on the battery storage problem, for the pre-trained MLP models.

The following files are assumed to exist. These files are created by running the commands listed in the README file under the heading, "Example: Battery storage CVaR control."

```
out/
  storage_mlp_shuffle/
    mlp_s{seed}.pt
```

This notebook creates the following files:

```
analysis/
  plots/
    storage_cvar_by_calibsize.pdf/.png
    storage_taskloss_by_calibsize.pdf/.png
out/
  storage_mlp_shuffle/
    crc_vary_calibsize.csv
```

In [ ]:
%load_ext autoreload
%autoreload 2

%cd ../

In [ ]:
from collections.abc import Iterable, Mapping
import itertools
import os

import numpy as np
import pandas as pd
import torch
from torch import Tensor
from tqdm.auto import tqdm

from models.mlp import MLP
from storage.data import get_tensors, get_train_calib_split
from storage.problems import (
    StorageConstants, StorageProblemNonRobust, StorageProblemLambda)
from run_storage import cvar, get_zs

Device = str | torch.device

INPUT_DIM = 101  # including future_temp
Y_DIM = 24
MAX_PRETRAIN_EPOCHS = 500
MAX_FINETUNE_EPOCHS = 100
BATCH_SIZE = 400
PSEUDOCAILB_SIZE = 200
SEEDS = range(10)
LOG_PRICES = False
LABEL_NOISE = 20

STORAGE_CONSTS = [
    StorageConstants(lam=0.1, eps=.05),
]

out_dir = 'out/storage_mlp_shuffle/'

savedir = 'analysis/plots'
os.makedirs(savedir, exist_ok=True)

In [ ]:
def get_lams(
    tensors_dict: Mapping[str, Tensor], model: MLP, prob: StorageProblemNonRobust,
    device: Device, alphas: Iterable[float], deltas: Iterable[float]
) -> pd.DataFrame:
    """
    Returns a DataFrame with columns:
        alpha, delta, lambda, t, task loss, CVaR
    """
    X_train = tensors_dict['X_train'].to(device, non_blocking=True)
    X_val = tensors_dict['X_calib'].to(device, non_blocking=True)
    Y_train_np = tensors_dict['Y_train'].cpu().numpy()
    Y_val_np = tensors_dict['Y_calib'].cpu().numpy()

    # get decision variables
    with torch.no_grad():
        model.eval().to(device)
        pred_np_train = model(X_train).cpu().numpy()
        pred_np_val = model(X_val).cpu().numpy()
    z_in_train, z_out_train, z_net_train = get_zs(prob, pred_np_train)
    z_in_val, z_out_val, z_net_val = get_zs(prob, pred_np_val)

    max_lambda_prob_var_t = StorageProblemLambda(
        T=Y_DIM, const=prob.const,
        y=Y_train_np, y_mean=prob.y_mean, y_std=prob.y_std,
        z_in=z_in_train, z_out=z_out_train, z_net=z_net_train, quad=False, t_fixed=False)

    max_lambda_prob_fixed_t = StorageProblemLambda(
        T=Y_DIM, const=prob.const,
        y=Y_val_np, y_mean=prob.y_mean, y_std=prob.y_std,
        z_in=z_in_val, z_out=z_out_val, z_net=z_net_val, quad=False, t_fixed=True)

    rows = []
    for alpha, delta in itertools.product(alphas, deltas):
        max_lambda_prob_var_t.solve(alpha, delta)
        assert max_lambda_prob_var_t.t.value is not None
        t = max_lambda_prob_var_t.t.value.item()

        λ = max_lambda_prob_fixed_t.solve(alpha, delta, t=t)
        if λ == 0.:
            val_task_loss = 0.
            val_cvar = 0.
        else:
            task_losses = prob.task_loss(
                z_in_val * λ, z_out_val * λ, z_net_val * λ,
                y=Y_val_np, is_standardized=True)
            financial_losses = prob.financial_loss(
                z_in_val * λ, z_out_val * λ, y=Y_val_np, is_standardized=True)
            assert isinstance(task_losses, np.ndarray)
            assert isinstance(financial_losses, np.ndarray)
            val_task_loss = np.mean(task_losses).item()
            val_cvar = cvar(financial_losses, q=delta)

        rows.append({
            'alpha': alpha, 'delta': delta,
            'lambda': λ,
            't': t,
            'task loss': val_task_loss,
            'CVaR': val_cvar
        })

    return pd.DataFrame(rows).set_index(['alpha', 'delta'])


def crc(
    shuffle: bool, future_temp: bool, label_noise: float, const: StorageConstants,
    alphas: Iterable[float], deltas: Iterable[float], seed: int, saved_ckpt_fmt: str,
    device: Device
) -> list[dict[str, float]]:
    """
    Post-hoc CRC. Always call this function within a torch.no_grad() context.

    Returns:
        list of dicts, one per (alpha, delta) pair, with keys:
            seed, alpha, delta, lambda, task loss, cvar
    """
    tensors, y_info = get_tensors(
        shuffle=shuffle, log_prices=LOG_PRICES, future_temp=future_temp,
        label_noise=label_noise)
    assert isinstance(y_info, tuple)
    y_mean, y_std = y_info
    tensors_cv, _ = get_train_calib_split(tensors, seed=seed)

    prob = StorageProblemNonRobust(T=Y_DIM, y_mean=y_mean, y_std=y_std, const=const)

    # load the model
    model = MLP(input_dim=tensors['X_test'].shape[1], y_dim=Y_DIM)
    saved_ckpt_path = saved_ckpt_fmt.format(seed=seed)
    model.load_state_dict(torch.load(saved_ckpt_path, weights_only=True))
    model.eval().to(device)

    rows = []

    calib_sizes = [None, 300, 100]
    for calib_size in calib_sizes:
        new_tensors_cv = tensors_cv.copy()
        if calib_size is not None:
            new_tensors_cv['X_calib'] = tensors_cv['X_calib'][:calib_size]
            new_tensors_cv['Y_calib'] = tensors_cv['Y_calib'][:calib_size]
        else:
            calib_size = len(new_tensors_cv['Y_calib'])

        calib_df = get_lams(
            tensors_dict=new_tensors_cv, model=model,
            prob=prob, device=device, alphas=alphas, deltas=deltas)

        # use lambdas on test set
        with torch.no_grad():
            model.eval().to(device)
            pred_np = model(tensors['X_test'].to(device)).cpu().numpy()  # type: ignore
        z_in, z_out, z_net = get_zs(prob, preds=pred_np)
        y_test = tensors['Y_test'].cpu().numpy()  # type: ignore
        for (alpha, delta) in calib_df.index:
            λ = calib_df.loc[(alpha, delta), 'lambda']
            t = calib_df.loc[(alpha, delta), 't']
            task_losses = prob.task_loss(z_in * λ, z_out * λ, z_net * λ, y=y_test, is_standardized=True)
            financial_losses = prob.financial_loss(z_in * λ, z_out * λ, y=y_test, is_standardized=True)
            assert isinstance(task_losses, np.ndarray)
            assert isinstance(financial_losses, np.ndarray)
            rows.append({
                'seed': seed, 'alpha': alpha, 'delta': delta, 'calib_size': calib_size,
                't': t, 'lambda': λ,
                'task loss': np.mean(task_losses).item(),
                'cvar': cvar(financial_losses, q=delta)
            })

    return rows

In [ ]:
all_rows: list[dict[str, float]] = []
saved_ckpt_fmt = os.path.join(out_dir, 'mlp_s{seed}.pt')
for s in tqdm(SEEDS):
    crc_results = crc(
        seed=s, shuffle=True, future_temp=False,
        label_noise=LABEL_NOISE, const=STORAGE_CONSTS[0],
        alphas=[2, 5, 10], deltas=[.9, .95, .99],
        saved_ckpt_fmt=saved_ckpt_fmt, device='cuda:0')
    all_rows.extend(crc_results)

# save results to file
df = pd.DataFrame(all_rows)

df.to_csv(os.path.join(out_dir, 'crc_vary_calibsize.csv'), index=False)

In [ ]:
df = pd.read_csv(os.path.join(out_dir, 'crc_vary_calibsize.csv'))
df = df.rename(columns={'delta': 'δ', 'alpha': 'α', 'lambda': 'λ'})
df

In [ ]:
df = df.set_index(['α', 'δ', 'calib_size', 'seed'])

In [ ]:
df

In [ ]:
summary = (
    df[['task loss', 'cvar']]
    .groupby(['α', 'δ', 'calib_size'])
    .agg(['mean', 'std'])
)

def mean_std(row: pd.Series, mean_fmt: str = '{:.2g}', std_fmt: str = '{:.2g}') -> str:
    mean = row['mean']
    std = row['std']
    if np.isnan(mean):
        assert np.isnan(std)
        return "nan"
    return f'{mean_fmt.format(mean)} ± {std_fmt.format(std)}'

In [ ]:
with pd.option_context('display.max_rows', 100):
    display(pd.DataFrame({
        'task loss': summary['task loss'].apply(mean_std, axis=1),
        'cvar': summary['cvar'].apply(mean_std, axis=1)
    }))

    fmt = '{:.1f}'
    kwargs = {'mean_fmt': fmt, 'std_fmt': fmt}

    print(
        summary['task loss'].apply(mean_std, axis=1, **kwargs).unstack(['α', 'δ']).to_latex()
    )
    print(
        summary['cvar'].apply(mean_std, axis=1, **kwargs).unstack(['α', 'δ']).to_latex()
    )

In [ ]:
df_long = df[['task loss', 'cvar']]
df_long.columns.name = 'metric'
df_long = df_long.stack().to_frame(name='value')
df_long

In [ ]:
import seaborn as sns
sns.set_style('darkgrid')

In [ ]:
g = sns.catplot(
    data=df_long, x='α', y='value', hue='calib_size',
    col='δ', row='metric',
    sharey=False, kind='box', height=2.8, aspect=1.4, 
    width=0.5
)

# Iterate through the axes and add vertical lines
for ax in g.axes.flatten():
    ticks = ax.get_xticks()
    for i in range(1, len(ticks)):
        ax.axvline((ticks[i] + ticks[i-1])/2, color="gray", linestyle="--", linewidth=1)

In [ ]:
ylabel = r'CVaR${}^\delta[L(\theta, \lambda)]$'
legend_title = 'calibration\nset size'
cvar_df = df_long.unstack('metric')[('value', 'cvar')].rename(ylabel).to_frame()
cvar_df.index = cvar_df.index.set_names(['α', 'δ', legend_title, 'seed'])

g_cvar = sns.catplot(
    data=cvar_df, x='α', y=ylabel, hue=legend_title, col='δ',
    sharey=True, kind='box', height=2.8,
    palette='Set2',
)

# center the legend title
assert g_cvar.legend is not None
g_cvar.legend.get_title().set_multialignment('center')

# Iterate through the axes and add vertical lines
for ax in g_cvar.axes.flatten():
    ticks = ax.get_xticks()
    for i in range(1, len(ticks)):
        ax.axvline((ticks[i] + ticks[i-1])/2, color="gray", linestyle="--", linewidth=1)

g_cvar.tight_layout(pad=0, w_pad=1.08)
g_cvar.figure.savefig(os.path.join(savedir, 'storage_cvar_by_calibsize.pdf'), pad_inches=0)
g_cvar.figure.savefig(os.path.join(savedir, 'storage_cvar_by_calibsize.png'), pad_inches=0, dpi=300)

In [ ]:
ylabel = 'task loss'
legend_title = 'calibration\nset size'
taskloss_df = df_long.unstack('metric')[('value', 'task loss')].rename(ylabel).to_frame()
taskloss_df.index = taskloss_df.index.set_names(['α', 'δ', legend_title, 'seed'])

g_taskloss = sns.catplot(
    data=taskloss_df, x='α', y=ylabel, hue=legend_title, col='δ',
    sharey=True, kind='box', height=2.8,
    palette='Set2',
)

# center the legend title
assert g_taskloss.legend is not None
g_taskloss.legend.get_title().set_multialignment('center')

# Iterate through the axes and add vertical lines
for ax in g_taskloss.axes.flatten():
    ticks = ax.get_xticks()
    for i in range(1, len(ticks)):
        ax.axvline((ticks[i] + ticks[i-1])/2, color="gray", linestyle="--", linewidth=1)

g_taskloss.tight_layout(pad=0, w_pad=1.08)
g_taskloss.figure.savefig(os.path.join(savedir, 'storage_taskloss_by_calibsize.pdf'), pad_inches=0)
g_taskloss.figure.savefig(os.path.join(savedir, 'storage_taskloss_by_calibsize.png'), pad_inches=0, dpi=300)

In [ ]:
import seaborn.objects as so

(
    so.Plot(df_long, x='calib_size', y='value', color='α')
    .add(so.Bar(), so.Agg(), so.Dodge())
    .facet(col='δ', row='metric')
)